<img src="logo.png">

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10, 10]
np.random.seed(42)

# Análisis de Componentes Principales (PCA)

En este notebook vamos a ver en primer lugar el algoritmo PCA de forma "manual", es decir, haremos los pasos uno por uno. Posteriormente veremos como usar la implementación de `scikit-learn`

### Ejemplo 1: 

En este ejemplo manual vamos a usar el dataset de flores Iris, y aplicaremos PCA para reducir su dimensionalidad de 3 a 2 dimensiones

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
iris.feature_names

In [ ]:
iris.data[:10]

In [ ]:
iris.target[:10]

Vamos a usar el endpoint de matplotlib notebook para poder modificar la orientación de un gráfico en 3d de las 3 dimensiones del dataset que vamos a utilizar 

In [ ]:
%matplotlib notebook

In [ ]:
from mpl_toolkits.mplot3d import Axes3D #es necesario importar esto para que se pueda usar la proyección 3d
import ipywidgets as widgets
from IPython.display import display


fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel("longitud sépalo", size=20)
ax.set_ylabel("anchura sépalo", size=20)
ax.set_zlabel("longitud pétalo", size=20)
ax.scatter(iris.data[:,0], iris.data[:,1], iris.data[:,2], c=iris.target,
           cmap=cm.prism)
ax.view_init(20, 120)
plt.show()

def actualizar_grafica(angulo1=20, angulo2=120):
    # Cambiamos el ángulo de la visualización
    ax.view_init(angulo1, angulo2)
    fig.canvas.draw_idle()

# widgets
angulo1_slider = widgets.IntSlider(20, min = 0, max = 90)
display(angulo1_slider)

def actualizar_angulo1(value):
    actualizar_grafica(angulo1=value['new'])

angulo1_slider.observe(actualizar_angulo1, names='value');

#### Paso 1 de PCA. Centrar los datos. 

Este paso consiste en restar a cada dimension (cada variable) su media. Vamos a eliminar la 4 variable del dataset para reducir de 3 dimensiones a 2 y poder hacerlo de forma gráfica

In [ ]:
iris_centrado = (iris.data - iris.data.mean(axis=0))[:,:3]

In [ ]:
iris_centrado[:10]

### Paso 2. Calcular la matriz de covarianza

Creamos la formula para la varianza y covarianza. Si se le pasa dos muestras devuelve la covarianza entre ambas variables, si se le pasa 1 devuelve la varianza de dicha variable.

Dado que generalmente vamos a trabajar con muestras (nunca con poblaciones completas) al calcular la varianza se divide la suma total de diferencias cuadraticas por N-1 en vez de por N. Esto se llama [Corrección de Bessel](https://es.wikipedia.org/wiki/Correcci%C3%B3n_de_Bessel) y se utiliza al calcular la desviación estandard o la varianza de una muestra para corregir el sesgo (bias) intrínseco de calcular estadísticos descriptivos tomando solo una porción de la población total.

In [ ]:
def varianza(var1, var2=None):
    if var2 is None:
        var2 = var1
    assert var1.shape== var2.shape
    var1_mean = var1.mean()
    var2_mean = var2.mean()
    return  np.sum((var1-var1_mean)*(var2-var2_mean)) / (var1.shape[0] -1)

In [ ]:
var1 = np.array([5,10,17,35])
var2 = np.array([34,70, 75,50])

In [ ]:
varianza(var1, var2)

In [ ]:
varianza(var1)

In [ ]:
varianza(var2)

En vez de calcular la varianza con nuestra función podemos usar la función de numpy `np.cov`

In [ ]:
np.cov(np.array([var1, var2]))

Calculamos ahora la matriz de covarianza de los datos centrados

In [ ]:
cov_mat = np.cov(m=iris_centrado.T)
cov_mat

### Paso 3. Descomponer matriz de covarianza
Ahora falta usar un método de álgebra lineal para obtener los vectores propios de dicha matriz de covarianza. Para ello podemos usar `np.linalg.eig` que descompone una matriz en sus vectores propios (eigenvectors)

In [ ]:
print(np.linalg.eig.__doc__)

In [ ]:
val_propios, vec_propios = np.linalg.eig(cov_mat)

print('Vectores Propios:\n', vec_propios)
print('\nValores Propios:', val_propios)

Los componentes principales son las columnas de la matriz de vectores propios

**OJO!**, en la implementación de `numpy.linalg.eig` los valores propios no estan ordenados de mayor a menor. Vemos que el valor propio 2 (el segundo en terminos de varianza) está en tercera posición en el vector.

Asi que podemos tomar los componentes principales y ordenarlos en función de la varianza explicada de cada uno

In [ ]:
np.argsort(val_propios)[::-1]

In [ ]:
orden_componentes = np.argsort(val_propios)[::-1]
val_propios_ordenados = val_propios[orden_componentes]

vec_propios_ordenados = vec_propios[:,orden_componentes]

print('Vectores Propios (ordenados):\n', vec_propios_ordenados)
print('\nValores Propios (ordenados):', val_propios_ordenados)

Los vectores propios son las coordenadas de los 3 vectores propios de la matriz de covarianza, es decir, los 3 componentes principales. Los 3 valores propios son la varianza que se explica mediante cada uno de los 3 componentes principales. Vemos que el primer componente es el más importante con diferencia.

Ahora vamos a hacer la misma gráfica en 3d, pero mostrando los componentes principales. Para mostrarlos como flechas, he copiado la implementación de [esta respuesta en StackOverflow](https://stackoverflow.com/a/22867877/1403840), donde se muestra una implementación de las flechas de matplotlib que funcionan en 3d.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

Los origenes de los componentes principales son las medias de las variables

In [ ]:
# las medias del dataset centrado son el origen de los componentes principales
media_x = iris_centrado[:,0].mean()
media_y = iris_centrado[:,1].mean()
media_z = iris_centrado[:,2].mean()

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel("eje X", size=20)
ax.set_ylabel("eje Y", size=20)
ax.set_zlabel("eje Z", size=20)

ax.scatter(iris_centrado[:,0], iris_centrado[:,1], iris_centrado[:,2], c=iris.target,
           cmap=cm.prism)

for v in vec_propios:
    
    a = Arrow3D([media_x, v[0]], [media_y, v[1]], 
                [media_z, v[2]], mutation_scale=20, 
                lw=3, arrowstyle="-|>", color="r")
    ax.add_artist(a)
    #ax.plot([media_x, v[0]], [media_y, v[1]], [media_z, v[2]], color='red', alpha=0.8, lw=3)
ax.view_init(20, 120)


def actualizar_grafica(angulo1=20, angulo2=120):
    # Cambiamos el ángulo de la visualización
    ax.view_init(angulo1, angulo2)
    fig.canvas.draw_idle()

# widgets
angulo1_slider = widgets.IntSlider(20, min = 0, max = 90)
display(angulo1_slider)

def actualizar_angulo1(value):
    actualizar_grafica(angulo1=value['new'])

angulo1_slider.observe(actualizar_angulo1, names = 'value')

Ahora para transformar puntos en el nuevo sistema de coordenadas de los componentes principales simplemente tenemos que multiplicar dichos puntos por el número de componentes principales que queramos considerar. 

#### ¿Cuantos componentes seleccionar?

En cuanto al número de componentes que elegir, podemos seguir ciertas reglas de sentido común:
- Elegir al menos componentes que sumen el 80% de la varianza total
- Elegir aquellos componentes con una varianza explicada mayor que la media
- Realizar un gráfico de los valores propios de cada componente de forma decreciente y usar el método del codo para ver en que momento hay ganancias decrecientes de los componentes principales (esto se llama gráfica *SCREE*).

In [ ]:
plt.plot(val_propios_ordenados)
plt.title("Gráfica SCREE de componentes principales");

Podemos ver el porcentaje de varianza que explica cada componente principal sumando los valores propios de cada vector propio y dividiendo por la suma total

In [ ]:
print("""
PCA 1: {0:.2f}% de la varianza
PCA 2:  {1:.2f}% de la varianza
PCA 3:  {2:.2f}% de la varianza
""".format(*tuple(val_propios_ordenados / val_propios_ordenados.sum() * 100)))

Por ejemplo, si queremos reducir el dataset de 3 dimensiones a 2, simplemente tenemos que tomar los dos primeros componentes principales (definidos como las columnas de la matriz de vectores propios).

In [ ]:
vec_propios_ordenados[:,:2].T

Ahora solo tenemos que hacer un producto de los datos con los componentes principales

In [ ]:
iris_coord_princ = iris_centrado @ vec_propios_ordenados[:,:2]

In [ ]:
iris_coord_princ[:10]

In [ ]:
fig = plt.figure(figsize=(12,12))
plt.scatter(iris_coord_princ[:,0], iris_coord_princ[:,1], c=iris.target, 
            cmap=cm.Set3)
plt.title("Dataset Iris descompuesto en sus 2 primeros componentes principales", size=18)
plt.xlabel("Componente Principal 1", size=18)
plt.ylabel("Componente Principal 2", size=18);

Ahora vemos como usar PCA en sklearn

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
iris_pca = pca.fit_transform(iris_centrado)

Podemos ver los componentes principales asi como su varianza explicada

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_ratio_

Vemos que en este caso el resultado de scikitlearn produce componentes principales con el sentido cambiado respecto a nuestra implementación "manual". Diversos métodos numéricos pueden producir los vectores en distinta orientación, esto no afecta al resultado (es decir, a la separación de los puntos en cuanto a su varianza).

In [ ]:
vec_propios[:,[0,2]].T

In [ ]:
iris_pca[:10]

In [ ]:
fig = plt.figure(figsize=(12,12))
plt.scatter(iris_pca[:,0], iris_pca[:,1], c=iris.target, 
            cmap=cm.Set3)
plt.title("Dataset Iris descompuesto en sus 2 primeros componentes principales (scikit-learn)", size=18)
plt.xlabel("Componente Principal 1", size=18)
plt.ylabel("Componente Principal 2", size=18);

### Ejemplo 2


En este ejemplo vamos a hacer la implementación de scikit learn con escalado.

Vamos a usar el dataset del cancer de mama (Breast Cancer dataset)

En primer lugar cargamos los datos:


In [ ]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [ ]:
cancer.data.shape

Posteriormente, tenemos que sustraerles la media, esto podemos hacerlo simplemente usando `sklearn.preprocessing.StandardScaler`, (recordad, el estandarizado resta la media y divide por la desviación estándar).

In [ ]:
from sklearn.preprocessing import StandardScaler

scalador = StandardScaler()
scalador.fit(cancer.data)
cancer_escalado = scalador.transform(cancer.data)

pca = PCA(n_components=2)
pca.fit(cancer_escalado)

cancer_pca = pca.transform(cancer_escalado)

In [ ]:
fig = plt.figure(figsize=(12,12))
plt.scatter(cancer_pca[:, 0], cancer_pca[:, 1], c=cancer.target, cmap=cm.Set3)
plt.title("Dataset Cáncer de mama 2 primeros componentes principales", size=18)
plt.xlabel("Componente Principal 1", size=18)
plt.ylabel("Componente Principal 2", size=18);

Vemos como reducir la dimensionalidad de 30 dimensiones a 2 componentes principales nos vale para separar la mayor parte de los cánceres en benignos y malignos.